<a href="https://colab.research.google.com/github/oscarserc/LENGUAJE_NLP/blob/main/Transformers_4_HuggingFace_Generacion_de_TEXTO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# -----------------------------------------------------------
# PRACTICAS DE: Natural Language Processing with Transformers  - CAPITULO 5
# -----------------------------------------------------------

# GitHub del libro: https://github.com/nlp-with-transformers/notebooks

# Uncomment and run this cell if you're on Colab or Kaggle
!git clone https://github.com/nlp-with-transformers/notebooks.git
%cd notebooks
from install import *
install_requirements()



Cloning into 'notebooks'...
remote: Enumerating objects: 422, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 422 (delta 0), reused 5 (delta 0), pack-reused 416
Receiving objects: 100% (422/422), 24.97 MiB | 11.98 MiB/s, done.
Resolving deltas: 100% (190/190), done.
/content/notebooks
⏳ Installing base requirements ...
✅ Base requirements installed!
⏳ Installing Git LFS ...
✅ Git LFS installed!


#**Empezamos por GREEDY SEARCH DECODING**.
#(La parte final no se puede completar debido a que se agota la memoria RAM.
#*Aconseja pasarse a Colab+*)

In [ ]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

DEVICE     = 'cuda' if torch.cuda.is_available() else 'cpu'
model_name = 'gpt2-xl'
tokenizer  = AutoTokenizer.from_pretrained(model_name)
model      = AutoModelForCausalLM.from_pretrained(model_name).to(DEVICE)


Downloading:   0%|          | 0.00/689 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.99G [00:00<?, ?B/s]

In [ ]:

import pandas as pd

input_txt = 'Transformers are the'
input_ids = tokenizer(input_txt, return_tensors='pt')['input_ids'].to(DEVICE)

iterations = []
n_steps    = 8
choices_per_step = 5

with torch.no_grad():
  for _ in range(n_steps):
    iteration = dict()
    iteration['Input'] = tokenizer.decode(input_ids[0])
    output             = model(input_ids=input_ids)
    # Seleccionamos LOGITS del primer BATCH y el último TOKEN, después aplicamos SOFTMAX
    next_token_logits = output.logits[0, -1, :]
    next_token_probs  = torch.softmax(next_token_logits, dim=-1)
    sorted_ids        = torch.argsort(next_token_probs, dim=-1, descending=True)
    # Almacenamos los TOKENS con la PROBABILIDAD MÁS ALTA
    for choice_idx in range(choices_per_step):
      token_id     = sorted_ids[choice_idx]
      token_prob   = next_token_probs[token_id].cpu().numpy()
      token_choice = (
          f'{tokenizer.decode(token_id)} ({100*token_prob:.2f}%)'
      )
      iteration[f'Choice {choice_idx+1}'] = token_choice
    # Anexamos el SIGUIENTE TOKEN PREDICHO a los INPUTS
    input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
    iterations.append(iteration)

pd.DataFrame(iterations)      
